### Npz files 1000 perm 

In [ ]:
from brainpipe.system import study
import numpy as np
from os import listdir,makedirs
from os.path import isfile, join,exists
import numpy as np
from itertools import product

st = study('Olfacto')
phase = 'odor'
conds = ['low','high']

path_classif = join(st.path, 'classified/Classif_TPSim_R_by_strength_odor/')
path_pow = join(st.path, 'feature/0_Power_Retrieval_EpiPerf_LowHigh/')
path2save = join(st.path, 'figure/0_Classif_Power_R_EpiPerf_LowHigh_1000perm_BBG/')
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','VACJ'] #'MICP'
rois_to_keep = ['ACC','HC','IFG','Ins','MFG','OFC','PHG','SFG','pPirT','Amg','Amg-PirT']

fnames = np.load(path_pow+'{}_{}_{}_bipo_sel_physFT_pow.npz'.format(subjects[0],phase,conds[0]))['fname']
freqs = [str(i)+'_'+freq for i,freq in zip(range(0,5),fnames)] #.decode("utf-8")
#freqs = fnames

## Create a npz file with all sources informations + POWER (subjects, coords, labels)
for id_freq,freq in enumerate(freqs):
    su_codes, s_xyz, s_channels,s_labels, s_elec, s_elec_pow0, s_elec_pow1 = np.array([]),np.array([]), np.array([]), np.array([]),np.array([]),np.array([]),np.array([])
    s_stats, s_stats_t0, s_da, s_perm = np.array([]),np.array([]),np.array([]),np.array([])
    s_BA, s_AAL, s_AAL_RL, s_MAI, s_MAI_RL = np.array([]),np.array([]),np.array([]),np.array([]),np.array([])
    print(freq)
    for i,su in enumerate(sorted(subjects)):
        print(su)
        mat0 = np.load(path_pow+'{}_{}_{}_bipo_sel_physFT_pow.npz'.format(su,
                            phase,conds[0]), allow_pickle=True)
        mat1 = np.load(path_pow+'{}_{}_{}_bipo_sel_physFT_pow.npz'.format(su,
                            phase,conds[1]),allow_pickle=True)
        pow0 = mat0['xpow'][:,:,20:47,:] #.swapaxes(0,1).swapaxes(2,3) #27:47
        pow1 = mat1['xpow'][:,:,20:47,:] #.swapaxes(0,1).swapaxes(2,3)
        print(mat0.files,'labels',mat0['labels'].shape, mat0['xpow'].shape)
        print('pow files shape', pow0.shape,pow1.shape)
        xyz_all = mat0['xyz']
        for elec in range(xyz_all.shape[0]):
            ### INFO electrodes for all subjects
            xyz = xyz_all[elec]
            aal = mat0['s_aal'][elec]
            aal_RL = aal[:-4]
            BA = mat0['s_BA'][elec]
            lab = mat0['Mai_RL'][elec]
            label = lab
            #label = 'pPirT' if lab in ['Amg-PirT'] else lab
            label_RL = mat0['Mai'][elec]
            channel = mat0['channels'][elec]
            if label in rois_to_keep:
                s_BA = np.vstack((s_BA, BA)) if np.size(s_BA) else BA
                su_codes = np.vstack((su_codes,su)) if np.size(su_codes) else su
                s_xyz = np.vstack((s_xyz, xyz)) if np.size(s_xyz) else xyz
                s_AAL = np.vstack((s_AAL, aal)) if np.size(s_AAL) else aal
                s_AAL_RL = np.vstack((s_AAL_RL, aal_RL)) if np.size(s_AAL_RL) else aal_RL    
                s_MAI = np.vstack((s_MAI,label_RL)) if np.size(s_MAI) else label_RL
                s_labels = np.vstack((s_labels, label)) if np.size(s_labels) else label
                s_channels = np.vstack((s_channels, channel)) if np.size(s_channels) else channel
                ### POWER values for 1 electrode for 1 freq for Good & Bad conditions
                elec_pow0 = pow0[id_freq,elec,:,:]
                elec_pow1 = pow1[id_freq,elec,:,:]
                #print(elec_pow0.shape, elec_pow1.shape)
                ## AVERAGE POWER power across trials + select values only AFTER odor perception (14 windows)
                elec_pow0 = np.mean(elec_pow0, axis=1)#0 for rest
                elec_pow1 = np.mean(elec_pow1, axis=1)#0 for rest
                s_elec_pow0 = np.vstack((s_elec_pow0, elec_pow0)) if np.size(s_elec_pow0) else elec_pow0
                s_elec_pow1 = np.vstack((s_elec_pow1, elec_pow1)) if np.size(s_elec_pow1) else elec_pow1
                s_elec = np.vstack((s_elec,str(elec))) if np.size(s_elec) else str(elec)
                ### STATS & DECODING ACCURACY for all electrodes
                da = np.mean(np.load(join(path_classif, freq+'/auc/'+su+'_auc_'+conds[0]+'_'+conds[1]+'_'+label+'_('+str(elec)+').npy')),axis=0)
                perm = np.load(join(path_classif, freq+'/auc/'+su+'_perm_'+label+'_('+str(elec)+').npy'))[np.newaxis]
                #da = np.mean(np.load(join(path_classif, freq+'/auc/'+su+'_auc_2classes_'+label+'_('+str(channel)+').npy')),axis=0)
                #perm = np.load(join(path_classif, freq+'/auc/'+su+'_perm_2classes_'+label+'_('+str(channel)+').npy'))[np.newaxis]
                ### AVERAGE stats across repetitions for da + select only values AFTER odor perception
                s_da = np.vstack((s_da, da)) if np.size(s_da) else da
                s_perm = np.vstack((s_perm, perm)) if np.size(s_perm) else perm
    print ('before', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,'da', s_da.shape, 
           'perm',s_perm.shape,'aal',s_AAL.shape,'BA',s_BA.shape)   
    su_codes = np.squeeze(su_codes)
    s_labels = np.squeeze(s_labels)
    s_AAL = np.squeeze(s_AAL)
    s_AAL_RL = np.squeeze(s_AAL_RL)
    s_BA = np.squeeze(s_BA)
    s_MAI = np.squeeze(s_MAI)
    s_channels = np.squeeze(s_channels)
    s_elec = np.squeeze(s_elec)
    s_da = np.squeeze(s_da)
    s_perm = np.squeeze(s_perm)
    print ('after', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,
           'da', s_da.shape, 'perm',s_perm.shape,'aal',s_AAL.shape, 'BA',s_BA.shape)
    su_coord_label = {'su_codes':su_codes, 's_xyz':s_xyz, 's_labels':s_labels,'s_channels':s_channels, 
                      's_elec':s_elec, 's_elec_pow0':s_elec_pow0,
                     's_elec_pow1':s_elec_pow1,'s_da':s_da,'s_perm':s_perm,'s_aal':s_AAL,
                      's_BA':s_BA, 's_aal_RL':s_AAL_RL,'s_Mai':s_MAI}
    file_source = 'All_subjects_sources_'+freq+'_'+phase+'_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
    if not exists(path2save):
        makedirs(path2save)
    np.savez(path2save+file_source,**su_coord_label)

### NPZ files 1000perm 1 point in time
Old style classif scripts (one file by elec and freq)

In [1]:
from brainpipe.system import study
import numpy as np
from os import listdir,makedirs
from os.path import isfile, join,exists
import numpy as np
from itertools import product
from brainpipe.statistics import perm_pvalue2level

st = study('Olfacto')
phase = 'odor'
conds = ['low','high']
val = 'tps'

method = 'by_cond'#'by_cond'
reps = ['_Encoding_EpiPerf_LowHigh_mean_3s_bsl/','_Retrieval_EpiPerf_LowHigh_mean_3s_bsl/']

path_classif = join(st.path, 'classified/0_Classif_Pow_sklearn_mean_{}_'+method+'_3s/')
path_pow = path.join(st.path, 'feature/{}/{}_odor_{}_common_renamed_OFC_HC_pow.npz')
path2save = join(st.path, 'figure/0_LDA_Pow_{}_'+method+'/')
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','VACJ']
freqs = ['theta','alpha','beta','gamma']

rois_to_keep = ['ACC','aHC','pHC','IFG','Ins',rois_to_keep'MFG','OFC_olf','PHG','SFG','pPirT','Amg']

## Create a npz file with all sources informations + POWER (subjects, coords, labels)
for id_freq,freq in enumerate(freqs):
    su_codes, s_xyz, s_channels,s_labels, s_elec, s_elec_pow0, s_elec_pow1 = np.array([]),np.array([]), np.array([]), np.array([]),np.array([]),np.array([]),np.array([])
    s_stats, s_stats_t0, s_da, s_perm = np.array([]),np.array([]),np.array([]),np.array([])
    s_BA, s_AAL, s_AAL_RL, s_MAI, s_MAI_RL = np.array([]),np.array([]),np.array([]),np.array([]),np.array([])
    print(id_freq,freq)
    for i,su in enumerate(sorted(subjects)):
        if val == 'tps':
            if method == 'by_cond':
                mat0 = np.load(path_pow+'TPS_spear_{}_cond_{}_{}.npz'.format(su,conds[0],
                                freq[2:]),allow_pickle=True)
                mat1 = np.load(path_pow+'TPS_spear_{}_cond_{}_{}.npz'.format(su,conds[1],
                                freq[2:]),allow_pickle=True)
                pow0, pow1 = mat0['tps'], mat1['tps']
            else:
                mat0 = np.load(path_pow+'TPS_spear_{}_odor_{}_{}.npz'.format(su,conds[0],
                                freq),allow_pickle=True)
                mat1 = np.load(path_pow+'TPS_spear_{}_odor_{}_{}.npz'.format(su,conds[1],
                                freq),allow_pickle=True)
                pow0, pow1 = mat0['TPS'], mat1['TPS']
            label_code = 'label'
            print(mat0.files,'labels',mat0[label_code].shape,'low',pow0.shape,'high',pow1.shape)
        elif val == 'xpow':
            mat0 = np.load(path_pow+'{}_{}_{}_bipo_sel_physFT_pow.npz'.format(su,phase,conds[0]))
            mat1 = np.load(path_pow+'{}_{}_{}_bipo_sel_physFT_pow.npz'.format(su,phase,conds[1],freq))
            pow0 = np.mean(mat0['xpow'][id_freq,:,27:47,:],axis=2) #.swapaxes(0,1).swapaxes(2,3)
            pow1 = np.mean(mat1['xpow'][id_freq,:,27:47,:],axis=2) #.swapaxes(0,1).swapaxes(2,3)
            label_code = 'Mai_RL'
            print(mat0.files,'Mai RL',mat0[label_code].shape, pow0.shape,pow1.shape)
        xyz_all, channels = mat0['xyz'], mat0['channel']
        for elec in range(xyz_all.shape[0]):
            ### INFO electrodes for all subjects
            su_codes = np.vstack((su_codes,su)) if np.size(su_codes) else su
            xyz = xyz_all[elec]
            s_xyz = np.vstack((s_xyz, xyz)) if np.size(s_xyz) else xyz
            lab = mat0[label_code][elec]
            label = 'pPirT' if lab in ['Amg','Amg-PirT'] else lab
            if label in rois_to_keep:
                s_labels = np.vstack((s_labels, label)) if np.size(s_labels) else label
                elec_pow0 = pow0[elec,:]
                elec_pow1 = pow1[elec,:]
                ## AVERAGE POWER power across trials + select values only AFTER odor perception (14 windows)
                elec_pow0 = np.mean(elec_pow0)
                elec_pow1 = np.mean(elec_pow1)
                s_elec_pow0 = np.vstack((s_elec_pow0, elec_pow0)) if np.size(s_elec_pow0) else elec_pow0
                s_elec_pow1 = np.vstack((s_elec_pow1, elec_pow1)) if np.size(s_elec_pow1) else elec_pow1
                s_elec = np.vstack((s_elec,str(elec))) if np.size(s_elec) else str(elec)
                ### STATS & DECODING ACCURACY for all electrodes
                da = np.mean(np.load(join(path_classif, freq+'/'+su+'_auc'+'_'+conds[0]+'_'+conds[1]+'_'+lab+'_('+str(elec)+').npy')),axis=0)
                perm = np.load(join(path_classif, freq+'/'+su+'_perm'+'_'+lab+'_('+str(elec)+').npy'))
                ### AVERAGE stats across repetitions for da + select only values AFTER odor perception
                s_da = np.vstack((s_da, da)) if np.size(s_da) else da
                s_perm = np.concatenate((s_perm, perm),axis=1) if np.size(s_perm) else perm
        th_0_05_perm = perm_pvalue2level(s_perm, p=0.05, maxst=True)
        th_0_01_perm = perm_pvalue2level(s_perm, p=0.01, maxst=True)
        print(su,freq,th_0_05_perm[0],th_0_01_perm[0])
    print ('before', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,'da', s_da.shape, 
           'perm',s_perm.shape)   
    s_elec_pow0 = np.squeeze(s_elec_pow0)
    s_elec_pow1 = np.squeeze(s_elec_pow1)
    su_codes = np.squeeze(su_codes)
    s_labels = np.squeeze(s_labels)
    s_elec = np.squeeze(s_elec)
    s_da = np.squeeze(s_da)
    s_perm = np.squeeze(s_perm)
    print ('after', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,
           'da', s_da.shape, 'perm',s_perm.shape)
    su_coord_label = {'su_codes':su_codes, 's_xyz':s_xyz, 's_labels':s_labels,
                      's_elec':s_elec, 's_elec_pow0':s_elec_pow0,
                     's_elec_pow1':s_elec_pow1,'s_da':s_da,'s_perm':s_perm}
    file_source = 'All_subjects_sources_'+freq+'_'+phase+'_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
    if not exists(path2save):
        makedirs(path2save)
    np.savez(path2save+file_source,**su_coord_label)

-> Olfacto loaded
0 0_theta
['tps', 'pval', 'label', 'channel', 'xyz'] labels (57,) low (57, 66) high (57, 66)


FileNotFoundError: [Errno 2] No such file or directory: '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/classified/Classif_TPSim_R_by_strength_by_cond/0_theta/CHAF_auc_low_high_SFG_(0).npy'

### TPSim npz file regrouping all subjects
New scripts (sklearn functions)

In [1]:
###################################################
x0,y0,z0 = 20, 30,-16 #VOI for olf OFC
x1,y1,z1 = -20, 32,-16 #VOI for olf OFC
x0_ins,y0_ins,z0_ins = 42.0, -7.3, 0.3 #VOI for olf Ins
x1_ins,y1_ins,z1_ins = -38, -4, 12 #VOI for olf Ins
rad = 11 #radius of the VOI
HC_lim = -26 #limit anterior/posterior part of the aHC

def rename_elecs(names,x,y,z):
    
    labels = ['pPirT' if lab in ['Amg-PirT'] else lab for lab in names]

    labels = ['aHC' if lab == 'HC' and y[i] > -26 else lab \
                                  for i,lab in enumerate(labels)]
    
    id_R_OFC = [1 if all([x0-rad<=round(x[i])<=x0+rad, y0-rad<=round(y[i])<=y0+rad,
                    z0-rad<=round(z[i])<=z0+rad]) else 0 for i in range(len(labels))]
    id_L_OFC = [1 if all([x1-rad<=round(x[i])<=x1+rad, y1-rad<=round(y[i])<=y1+rad,
                    z1-rad<=round(z[i])<=z1+rad]) else 0 for i in range(len(labels))]
    idx_OFC = id_R_OFC + id_L_OFC
    labels = ['OFC_olf' if i == 1 else lab for i,lab in zip(idx_OFC,labels)]
    
    id_R_ins = [1 if all([x0_ins-rad<=round(x[i])<=x0_ins+rad, y0_ins-rad<=round(y[i])<=y0_ins+rad,
                    z0_ins-rad<=round(z[i])<=z0_ins+rad]) else 0 for i in range(len(labels))]
    id_L_ins = [1 if all([x1_ins-rad<=round(x[i])<=x1_ins+rad, y1_ins-rad<=round(y[i])<=y1_ins+rad,
                    z1_ins-rad<=round(z[i])<=z1_ins+rad]) else 0 for i in range(len(labels))]
    idx_ins = id_R_ins + id_L_ins
    labels = ['Ins_olf' if i == 1 else lab for i,lab in zip(idx_ins,labels)]

    return np.array(labels)

In [8]:
from brainpipe.system import study
import numpy as np
from os import listdir, makedirs
from os.path import isfile, join, exists
import numpy as np
from itertools import product
from brainpipe.statistics import perm_pvalue2level

st = study('Olfacto')
conds = ['low','high']
val = 'tps'
method = 'by_cond'#'by_cond'
reps = ['E','R']

rep_pow = join(st.path,'feature/TPSim_power_data/Power_all_elecs_E_R_by_cond/')
path_pow = join(rep_pow, '{}_odor_{}_{}_bsl_4freqs.npz')
path_classif = join(st.path, 'classified/0_clf_pow_sklearn_time_{}_by_cond/')
clf_name = join(path_classif, '{}_LDA_clf_{}_low_high_bsl.npz')
path2save = join(st.path, 'figure/0_clf_pow_sklearn_time_{}_'+method+'/')
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','VACJ']
rois_to_keep = ['ACC','aHC','IFG','Ins','MFG','OFC_olf','PHG','SFG','pPirT','pHC','Amg','HC']
freqs = ['theta','alpha','beta','BBG']

for rep in reps:
    ## Create a npz file with all sources informations + POWER (subjects, coords, labels)
    for id_freq,freq in enumerate(freqs):
        su_codes, s_xyz, s_channels,s_labels = np.array([]),np.array([]),np.array([]),np.array([])
        s_elec_pow0, s_elec_pow1,s_da = np.array([]),np.array([]), np.array([])
        s_elec, s_perm = np.array([]),np.array([])
        for i,su in enumerate(sorted(subjects)):
            if method != 'by_odor':
                pow0 = np.load(path_pow.format(su,conds[0],rep))[val][id_freq,:,17:47,:]
                pow1 = np.load(path_pow.format(su,conds[1],rep))[val][id_freq,:,17:47,:]
            classif = np.load(clf_name.format(rep,su,freq),allow_pickle=True)
    #         print(su, 'labels',classif['names'].shape,'low',pow0.shape,'high',pow1.shape)

            xyz_all, channels = classif['xyz'], classif['channels']
            auc, names, perm = classif['auc'], classif['names'], classif['perm']
            for elec in range(xyz_all.shape[0]):
                ### INFO electrodes for all subjects
                label = 'pPirT' if names[elec] in ['Amg','Amg-PirT'] else names[elec]
                chan = channels[elec]
                if label in rois_to_keep:
                    su_codes = np.vstack((su_codes,su)) if np.size(su_codes) else su
                    s_xyz = np.vstack((s_xyz, xyz_all[elec])) if np.size(s_xyz) else xyz_all[elec]
                    s_labels = np.vstack((s_labels, label)) if np.size(s_labels) else label
                    s_channels = np.vstack((s_channels, chan)) if np.size(s_channels) else chan
                    ## AVERAGE POWER power across trials + select values only AFTER odor perception (14 windows)
                    elec_pow0 = np.mean(pow0[elec,...],axis=-1)
                    elec_pow1 = np.mean(pow1[elec,...],axis=-1)
                    s_elec_pow0 = np.vstack((s_elec_pow0, elec_pow0)) if np.size(s_elec_pow0) else elec_pow0
                    s_elec_pow1 = np.vstack((s_elec_pow1, elec_pow1)) if np.size(s_elec_pow1) else elec_pow1
                    ### STATS & DECODING ACCURACY for all electrodes
                    elec_auc, elec_perm = auc[elec], perm[elec][np.newaxis]
                    s_da = np.vstack((s_da, elec_auc)) if np.size(s_da) else elec_auc
                    s_perm = np.concatenate((s_perm, elec_perm),axis=0) if np.size(s_perm) else elec_perm
                    s_elec = np.vstack((s_elec,str(elec))) if np.size(s_elec) else str(elec)
        print ('before', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,'da', s_da.shape, 
               'perm',s_perm.shape)
        s_elec_pow0 = np.squeeze(s_elec_pow0)
        s_elec_pow1 = np.squeeze(s_elec_pow1)
        su_codes = np.squeeze(su_codes)
        s_labels = np.squeeze(s_labels)
        s_channels = np.squeeze(s_channels)
        s_da = np.squeeze(s_da)
        s_perm = np.squeeze(s_perm)
        s_elec = np.squeeze(s_elec)
        print ('after', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,
               'da', s_da.shape, 'perm',s_perm.shape,'elec',s_elec.shape)
        su_coord_label = {'su_codes':su_codes, 's_xyz':s_xyz, 's_labels':s_labels,
                          's_elec_pow0':s_elec_pow0,'s_elec_pow1':s_elec_pow1,
                          's_da':s_da,'s_perm':s_perm,'s_elec':s_elec,'s_channels':s_channels}
        file_source = 'All_subjects_sources_'+freq+'_'+conds[0]+'_'+conds[1]+'_bsl_3s.npz'
        if not exists(path2save.format(rep)):
            makedirs(path2save.format(rep))
        np.savez(path2save.format(rep)+file_source,**su_coord_label)


-> Olfacto loaded
before pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000)
after pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000) elec (247,)
before pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000)
after pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000) elec (247,)
before pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000)
after pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000) elec (247,)
before pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000)
after pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000) elec (247,)
before pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000)
after pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000) elec (247,)
before pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000)
after pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30, 1000) elec (247,)
before pow1 (247, 30) pow0 (247, 30) da (247, 30) perm (247, 30,

### Npz PAC files

In [ ]:
from brainpipe.system import study
import numpy as np
from os import listdir,makedirs
from os.path import isfile, join,exists
import numpy as np
from itertools import product

st = study('Olfacto')
phase = 'odor'
conds = ['Low','High']

path_classif = join(st.path, 'classified/2_Classif_PAC_Encoding_EpiPerf_LowHigh_1000perm/')
path_pow = join(st.path, 'feature/2_PAC_Encoding_EpiPerf_LowHigh/')
path2save = join(st.path, 'figure/LDA_PAC_E_EpiPerf_LowHigh_BBG/')
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','VACJ'] #'MICP'

fnames = np.load(path_pow+'{}_{}_{}_bipo_sel_physFT_pac.npz'.format(subjects[0],phase,conds[0].lower()))['fname']
freqs = [str(i)+'_'+freq for i,freq in zip(range(0,5),fnames)] #.decode("utf-8")
famp = '0_gamma'

## Create a npz file with all sources informations + POWER (subjects, coords, labels)
for id_freq,freq in enumerate(freqs):
    su_codes, s_xyz, s_channels,s_labels, s_elec, s_elec_pow0, s_elec_pow1 = np.array([]),np.array([]), np.array([]), np.array([]),np.array([]),np.array([]),np.array([])
    s_stats, s_stats_t0, s_da, s_perm = np.array([]),np.array([]),np.array([]),np.array([])
    s_BA, s_AAL, s_AAL_RL, s_MAI_RL = np.array([]),np.array([]),np.array([]),np.array([])
    print(freq)
    for i,su in enumerate(sorted(subjects)):
        print(su)
        mat0 = np.load(path_pow+'{}_{}_{}_bipo_sel_physFT_pac.npz'.format(su,phase,conds[0].lower()))
        mat1 = np.load(path_pow+'{}_{}_{}_bipo_sel_physFT_pac.npz'.format(su,phase,conds[1].lower()))
        pow0 = mat0['pac'][:,:,27:47,:] #.swapaxes(0,1).swapaxes(2,3)
        pow1 = mat1['pac'][:,:,27:47,:] #.swapaxes(0,1).swapaxes(2,3)
        print(mat0.files,'labels',mat0['Mai_RL'].shape, mat0['pac'].shape)
        print('pow files shape', pow0.shape,pow1.shape)
        xyz_all = mat0['xyz']
        for elec in range(xyz_all.shape[0]):
            ### INFO electrodes for all subjects
            su_codes = np.vstack((su_codes,'S'+str(i))) if np.size(su_codes) else 'S'+str(i)
            xyz = xyz_all[elec]
            s_xyz = np.vstack((s_xyz, xyz)) if np.size(s_xyz) else xyz
            aal = mat0['s_aal'][elec]
            s_AAL = np.vstack((s_AAL, aal)) if np.size(s_AAL) else aal
            aal_RL = aal[:-4]
            s_AAL_RL = np.vstack((s_AAL_RL, aal_RL)) if np.size(s_AAL_RL) else aal_RL
            BA = mat0['s_BA'][elec]
            s_BA = np.vstack((s_BA, BA)) if np.size(s_BA) else BA
            label = mat0['Mai_RL'][elec]
            channel = mat0['channels'][elec]         
            s_labels = np.vstack((s_labels, label)) if np.size(s_labels) else label
            s_channels = np.vstack((s_channels, channel)) if np.size(s_channels) else channel
            ### POWER values for 1 electrode for 1 freq for Good & Bad conditions
            elec_pow0 = pow0[id_freq,elec,:,:]
            elec_pow1 = pow1[id_freq,elec,:,:]
            #print(elec_pow0.shape, elec_pow1.shape)
            ## AVERAGE POWER power across trials + select values only AFTER odor perception (14 windows)
            elec_pow0 = np.mean(elec_pow0, axis=1)#0 for rest
            elec_pow1 = np.mean(elec_pow1, axis=1)#0 for rest
            s_elec_pow0 = np.vstack((s_elec_pow0, elec_pow0)) if np.size(s_elec_pow0) else elec_pow0
            s_elec_pow1 = np.vstack((s_elec_pow1, elec_pow1)) if np.size(s_elec_pow1) else elec_pow1
            s_elec = np.vstack((s_elec,str(elec))) if np.size(s_elec) else str(elec)
            ### STATS & DECODING ACCURACY for all electrodes
            da = np.mean(np.load(join(path_classif, famp+'/'+freq+'/auc/'+su+'_auc_'+conds[0]+'_'+conds[1]+'_'+label+'_('+str(elec)+').npy')),axis=0)
            perm = np.load(join(path_classif, famp+'/'+freq+'/auc/'+su+'_perm_'+conds[0]+'_'+conds[1]+'_'+label+'_('+str(elec)+').npy'))[np.newaxis]
            ### AVERAGE stats across repetitions for da + select only values AFTER odor perception
            s_da = np.vstack((s_da, da)) if np.size(s_da) else da
            s_perm = np.vstack((s_perm, perm)) if np.size(s_perm) else perm
    print ('before', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,'da', s_da.shape, 
           'perm',s_perm.shape,'aal',s_AAL.shape,'BA',s_BA.shape)   
    su_codes = np.squeeze(su_codes)
    s_labels = np.squeeze(s_labels)
    s_AAL = np.squeeze(s_AAL)
    s_AAL_RL = np.squeeze(s_AAL_RL)
    s_BA = np.squeeze(s_BA)
    s_channels = np.squeeze(s_channels)
    s_elec = np.squeeze(s_elec)
    s_da = np.squeeze(s_da)
    s_perm = np.squeeze(s_perm)
    print ('after', 'pow1',s_elec_pow1.shape, 'pow0',s_elec_pow0.shape,
           'da', s_da.shape, 'perm',s_perm.shape,'aal',s_AAL.shape, 'BA',s_BA.shape)
    su_coord_label = {'su_codes':su_codes, 's_xyz':s_xyz, 's_labels':s_labels,'s_channels':s_channels, 
                      's_elec':s_elec, 's_elec_pow0':s_elec_pow0,
                     's_elec_pow1':s_elec_pow1,'s_da':s_da,'s_perm':s_perm,'s_aal':s_AAL,
                      's_BA':s_BA, 's_aal_RL':s_AAL_RL}
    file_source = 'All_subjects_sources_'+freq+'_'+phase+'_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
    if not exists(path2save):
        makedirs(path2save)
    np.savez(path2save+file_source,**su_coord_label)